In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

In [5]:
raw_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Lotte/final_purchase_data+weather 2.csv', encoding='cp949')

In [7]:
df = raw_df

In [8]:
df

,고객번호,채널구분,제휴사코드,제휴사세부코드,상품코드,구매일자,구매시간,구매금액,구매수량,성별,...,점포위치세부코드,구매수량당_구매금액,요일,거주_점포일치여부,Date,평균기온(°C),평균 상대습도(%),일강수량(mm),idx,대분류명_code
0,M001444118,1,103,1030260,1156,20210101,18,2650.0,1,0,...,1018,2650.0,4,0,2021-01-01,-4.2,64.0,0.0,365,1
1,M001444118,1,103,1030260,378,20210101,18,5990.0,1,0,...,1018,5990.0,4,0,2021-01-01,-4.2,64.0,0.0,365,1
2,M001444118,1,103,1030260,1160,20210101,18,4990.0,1,0,...,1018,4990.0,4,0,2021-01-01,-4.2,64.0,0.0,365,1
3,M001444118,1,103,1030260,223,20210101,18,1290.0,1,0,...,1018,1290.0,4,0,2021-01-01,-4.2,64.0,0.0,365,2
4,M003827618,1,103,1030295,89,20210101,14,5000.0,1,0,...,1018,5000.0,4,1,2021-01-01,-4.2,64.0,0.0,365,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381738,M744059040,1,102,1020116,318,20211231,14,8480.0,1,0,...,17002,8480.0,4,0,2021-12-31,4.9,62.9,0.0,4744,1
4381739,M744059040,1,102,1020116,395,20211231,14,3800.0,1,0,...,17002,3800.0,4,0,2021-12-31,4.9,62.9,0.0,4744,3
4381740,M803637343,1,102,1020116,1437,20211231,21,1990.0,1,1,...,17002,1990.0,4,0,2021-12-31,4.9,62.9,0.0,4744,1
4381741,M803637343,1,102,1020116,1422,20211231,21,2580.0,2,1,...,17002,1290.0,4,0,2021-12-31,4.9,62.9,0.0,4744,1


In [9]:
df.columns

Index(['고객번호', '채널구분', '제휴사코드', '제휴사세부코드', '상품코드', '구매일자', '구매시간', '구매금액',
       '구매수량', '성별', '연령대', '거주지대분류코드', '대분류명', '점포위치코드', '점포위치세부코드',
       '구매수량당_구매금액', '요일', '거주_점포일치여부', 'Date', '평균기온(°C)', '평균 상대습도(%)',
       '일강수량(mm)', 'idx', '대분류명_code'],
      dtype='object')

In [10]:
temp_df1 = df[['고객번호', '대분류명_code']] # 채널구분 추출을 위해서는 ['고객번호', '채널구분']으로 데이터프레임 생성후 진행

In [11]:
temp_df1

,고객번호,대분류명_code
0,M001444118,1
1,M001444118,1
2,M001444118,1
3,M001444118,2
4,M003827618,1
...,...,...
4381738,M744059040,1
4381739,M744059040,3
4381740,M803637343,1
4381741,M803637343,1


In [12]:
l = temp_df1.groupby('고객번호')

In [13]:
cust_list = l.var().index

### 예시

- 고객번호 M001444118의 대분류 code를 뽑아본다.

In [14]:
str_expr = '고객번호 == "M001444118"'

tempdf = temp_df1.query(str_expr)

In [15]:
tempdf

,고객번호,대분류명_code
0,M001444118,1
1,M001444118,1
2,M001444118,1
3,M001444118,2
31688,M001444118,8
...,...,...
4325370,M001444118,1
4325371,M001444118,1
4325372,M001444118,2
4325373,M001444118,1


In [16]:
def modefinder(numbers):   #numbers는 리스트나 튜플 형태의 데이터
    c = Counter(numbers)
    mode = c.most_common()
    return mode[0][0]

modefinder(tempdf['대분류명_code']) # 채널구분 추출을 위해 tempdf['채널구분 입력']

1

In [18]:
def find_max(df) :
    n = []
    
    for i in range(len(cust_list)) : 
        n.append(0)
        str_expr = '고객번호 == "{}"'.format(cust_list[i])
        tempdf = df.query(str_expr)
        n[i] = modefinder(tempdf['대분류명_code']) # 채널구분을 추출하고 싶으면 대분류_code대신 채널구분을 입력하면 추출 가능
        print(cust_list[i], n[i], i)
        
    df_ = pd.DataFrame({'고객번호' : cust_list,
                     '대분류명_mode' : n}) # '채널구분'으로 변경
    return df_

temp_df = find_max(temp_df1)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
M810385081 6 21917
M810404844 1 21918
M810427322 1 21919
M810434604 5 21920
M810526566 1 21921
M810527166 1 21922
M810528175 6 21923
M810565870 8 21924
M810570492 1 21925
M810594696 1 21926
M810628573 6 21927
M810680076 6 21928
M810730251 1 21929
M810767211 2 21930
M810774260 1 21931
M810825431 2 21932
M810847364 1 21933
M810872992 6 21934
M810881357 1 21935
M810903777 2 21936
M810911194 7 21937
M810948016 2 21938
M811012878 2 21939
M811026619 1 21940
M811071412 7 21941
M811079649 2 21942
M811137208 2 21943
M811196912 9 21944
M811221360 6 21945
M811243579 6 21946
M811289103 8 21947
M811477158 1 21948
M811510101 6 21949
M811578017 1 21950
M811619354 1 21951
M811666510 2 21952
M811831696 1 21953
M811927822 1 21954
M811975071 1 21955
M811978980 2 21956
M812022645 6 21957
M812064169 1 21958
M812078709 6 21959
M812082588 1 21960
M812128849 2 21961
M812131352 9 21962
M812149017 1 21963
M812171121 6 21964
M812176023 1 21965
M812221660 1 21966
M812269134 2 2

In [19]:
temp_df.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Lotte/고객번호_대분류.csv', index=False)